# Post processing reference data collected using CEO

Collect Earth Online is being used as a tool for collecting cropland reference data, some of the points are labelled as 'mixed' or 'unsure', and others are GFSAD validation points so we need to seperate these from the dataset. We also need to seperate a validation dataset for use in the final map accuracy assessment.

Inputs will be:

1. `ceo-data....csv` : The results from collecting training data in the CEO tool

Output will be:
1. Validation crop/non-crop samples as a shapefile
2. Training crop/non-crop samples as a shapefile
3. 'Mixed' samples as a shapefile (for use in manually copllecting more training data)
4. Crop/non-crop samples to send to RE for use in validating our data collection

***

*Filename guide:*

* `<aez>_training_data.geojson`: The final training dataset that includes CEO data, manually collected polygons, and any pre-existing datasets.
* `validation_samples.shp`: Validaiton data, collected using CEO, isolated from training data and only to be used for final map validation
* `mixed_samples_points(polys).shp`: The samples labelled as 'mixed' during the CEO process, these are used for guiding manual polygon drawing to increase the amount of training data e.g. using ArcGIS
* `ceo_td_polys.geojson` : training data polygons retrievd from CEO tool, these are combined the manually collected polygons and any pre-existing datasets


In [3]:
import pandas as pd
import numpy as np
import seaborn as sn
import geopandas as gpd
import matplotlib.pyplot as plt

## Analysis Parameters

In [2]:
folder = 'data/training_validation/collect_earth/indian_ocean/'
# csv = 'data/training_validation/collect_earth/indian_ocean/ceo-Cropland-Reference-Data-Acquisition---Indian-Ocean-Region---Ken-sample-data-2020-12-09.csv'
csv= 'data/training_validation/collect_earth/indian_ocean/ceo-GFSAD-divergent-samples-sample-data-2020-12-10.csv'

### Load the dataset

In [3]:
#ground truth shapefile
df = pd.read_csv(csv)

In [ ]:
df.columns

### Clean up dataframe


In [4]:
#This line if entire dataset:
df = df[['lon', 'lat', 'smpl_sampleid', 'smpl_gfsad_samp','smpl_class','Is the sample area entirely: crop, non-crop, mixed, or unsure?']]

# df= df[['lon', 'lat', 'smpl_sampleid', 'smpl_gfsad_clas','smpl_ken_class','Is the sample area entirely: crop, non-crop, mixed, or unsure?']]

#rename columns
df = df.rename(columns={'Is the sample area entirely: crop, non-crop, mixed, or unsure?':'Class'})

#remove nan rows
df = df.dropna()
print(len(df))
df.head()

123


,lon,lat,smpl_sampleid,smpl_gfsad_clas,smpl_ken_class,Class
0,46.883703,-23.151202,1,1,0,non-crop
1,46.254703,-22.720549,2,1,0,unsure
2,46.153373,-22.528130,3,1,0,mixed
3,45.489339,-25.379383,4,1,0,non-crop
4,47.384963,-23.183002,5,1,0,non-crop


### Remove GFSAD validation points


In [ ]:
df = df[df['smpl_gfsad_samp']==False]
print(len(df))

***

### Seperate 'Mixed' & 'Unsure' samples

In [5]:
#seperate class
mixed = df[df['Class']=='mixed']
unsure = df[df['Class']=='unsure']
crop = df[df['Class']=='crop']
noncrop = df[df['Class']=='non-crop']

print('Number of samples labelled as mixed: '+str(len(mixed)))
print('Number of samples labelled as unsure: ' + str(len(unsure)))
print('Number of samples labelled as crop: ' + str(len(crop)))
print('Number of samples labelled as non-crop: ' + str(len(noncrop)))


Number of samples labelled as mixed: 29
Number of samples labelled as unsure: 16
Number of samples labelled as crop: 14
Number of samples labelled as non-crop: 64


In [ ]:
# export mixed samples as shapefile
mixed = gpd.GeoDataFrame(
    mixed, geometry=gpd.points_from_xy(mixed.lon, mixed.lat, crs='EPSG:4326'))

radius = 20

#convert to equal area to set polygon size in metres
mixed_poly = mixed.to_crs('EPSG:6933')

#create circle buffer around points, then find envelope
mixed_poly['geometry'] = mixed_poly['geometry'].buffer(radius).envelope

#Convert back to lat/lon
mixed_poly = mixed_poly.to_crs('EPSG:4326')

#shuffle so labels aren't sequential
mixed_poly = mixed_poly.reset_index(drop=True).drop(columns=['lon', 'lat', 'smpl_gfsad_samp'])

#add plotid for CEO
mixed_poly['PLOTID'] = range(1,len(mixed_poly)+1)
mixed_poly['SAMPLEID'] = range(1,len(mixed_poly)+1)

mixed_poly.to_file(folder + 'mixed_samples_poly.shp')
mixed.to_file(folder + 'mixed_samples_points.shp')

## Sample to send to Radiant Earth

Randomly select 50 crop and 50 non-crop samples to send to Radiant Earth.

In [ ]:
#sample 50 points from each class
re_crop = crop.sample(n=50, replace=False).reset_index(drop=True)
re_noncrop = noncrop.sample(n=50, replace=False).reset_index(drop=True)

#merge dfs
re_sample = pd.concat([re_crop,re_noncrop]).reset_index(drop=True).drop(columns=['smpl_gfsad_samp'])

re_sample = gpd.GeoDataFrame(
    re_sample, geometry=gpd.points_from_xy(re_sample.lon, re_sample.lat, crs='EPSG:4326')).drop(columns=['lon', 'lat'])

#set radius (in metres) around points
radius = 20

#convert to equal area to set polygon size in metres
re_sample_poly = re_sample.to_crs('EPSG:6933')

#create circle buffer around points, then find envelope
re_sample_poly['geometry'] = re_sample_poly['geometry'].buffer(radius).envelope

#Convert back to lat/lon
re_sample_poly = re_sample_poly.to_crs('EPSG:4326')

#shuffle so labels aren't sequential
re_sample_poly = re_sample_poly.sample(frac=1).reset_index(drop=True)

#export to geojson
re_sample_poly.to_file(folder+'Eastern_region_RE_sample.geojson', driver='GeoJSON')

## Isolate samples as a validation dataset

In [ ]:
#create ramdom samples
validation_crop = crop.sample(n=100)
validation_noncrop = noncrop.sample(n=200)

#remove samples from crop/non-crop datasets
crop = crop.drop(validation_crop.index)
noncrop = noncrop.drop(validation_noncrop.index)

#join as a single dataset
validation_df = pd.concat([validation_crop, validation_noncrop]).sample(frac=1).reset_index(drop=True)

#create shapefile
validation_df = gpd.GeoDataFrame(
    validation_df, geometry=gpd.points_from_xy(validation_df.lon, validation_df.lat, crs='EPSG:4326'))

#export to file
validation_df.to_file(folder + 'validation_samples.shp')

#plot
validation_df.plot(column='Class', figsize=(6,6), cmap='viridis', legend=True)
plt.title('Validation Points');

## Process remaining points as training data

If collecting more data manually using the 'mixed' samples, then join that shapefile with this dataset after running the code below

In [ ]:
td = pd.concat([crop,noncrop]).reset_index(drop=True).drop(columns=['smpl_gfsad_samp'])

td = gpd.GeoDataFrame(
    td, geometry=gpd.points_from_xy(td.LON, td.LAT, crs='EPSG:4326')).drop(columns=['LON', 'LAT'])

#set radius (in metres) around points
radius = 20

#convert to equal area to set polygon size in metres
td_poly = td.to_crs('EPSG:6933')

#create circle buffer around points, then find envelope
td_poly['geometry'] = td_poly['geometry'].buffer(radius).envelope

#Convert back to lat/lon
td_poly = td_poly.to_crs('EPSG:4326')

#shuffle so labels aren't sequential
td_poly = td_poly.sample(frac=1).reset_index(drop=True).drop(columns=['smpl_sampleid'])


In [ ]:
td_poly['Class'] = np.where(td_poly['Class']=='non-crop', 0, td_poly['Class'])
td_poly['Class'] = np.where(td_poly['Class']=='crop', 1, td_poly['Class'])

In [ ]:
td_poly.to_file(folder + 'ceo_td_polys.geojson', driver='GeoJSON')

In [ ]:
td.plot(column='Class', figsize=(6,6), cmap='viridis', legend=True)
plt.title('Training Data Points');

## Optional: Join training data with other data

e.g. manually collected data and/or pre-existing datasets

In [14]:
folder = 'data/training_validation/collect_earth/eastern/'

In [20]:
#add manually collected polygons
manual = gpd.read_file(folder+'eastern_manual_crop_polys.shp')#.drop(['SMPL_SAMPL'],axis=1)
manual['Class'] = np.where(manual['Class']=='crop', 1, manual['Class'])
manual['Class'] = np.where(manual['Class']=='non-crop', 0, manual['Class'])

In [21]:
#sample the radiant earth dataset
re = gpd.read_file(folder+'ref_eastafrican_crops.geojson')

num_of_samples=len(re)

re = re.sample(n=num_of_samples).drop(['Estimated Harvest Date', 'id'], axis=1)
re['Class'] = 1

In [22]:
#open Collect Earth td polys
td_poly=gpd.read_file(folder + 'Eastern_ceo_training_samples.geojson')

In [23]:
training_data = pd.concat([manual,re,td_poly]).reset_index(drop=True)

In [24]:
training_data['Class'] = training_data['Class'].astype(int)

In [25]:
print('Crop samples = '+str(len(training_data[training_data['Class']==1])))
print('Non-Crop samples = '+str(len(training_data[training_data['Class']==0])))

Crop samples = 1951
Non-Crop samples = 2013


In [26]:
training_data.to_file(folder+'Eastern_training_data_20201215.geojson', driver='GeoJSON')

In [ ]:
training_data

## Use this to find difference between analyst's lables and the original GFSAD stratification label

In [ ]:
# df['Class'] = np.where(df['Class']=='non-crop', 0, df['Class'])
# df['Class'] = np.where(df['Class']=='crop', 1, df['Class'])

# df['smpl_class'] = np.where(df['smpl_class']==1, 0, df['smpl_class'])
# df['smpl_class'] = np.where(df['smpl_class']==2, 1, df['smpl_class'])

# df = df.drop(df[df['Class']=='mixed'].index)
# df = df.drop(df[df['Class']=='unsure'].index)

# df=df[df['smpl_class']!= df['Class']]

# df=df.rename(columns={'smpl_class':'gfsad_class', 'Class': 'ken_class', 'smpl_sampleid':'original_sampleid'}).drop(columns=['smpl_gfsad_samp'])

# crop=df[df['gfsad_class']==1].sample(n=100, replace=False).reset_index(drop=True)
# noncrop = df[df['gfsad_class']==0].reset_index(drop=True) #grab all points

# #merge dfs
# df = pd.concat([crop,noncrop]).reset_index(drop=True)

# gdf = gpd.GeoDataFrame(
#     df, geometry=gpd.points_from_xy(df.lon, df.lat, crs='EPSG:4326'))

# radius = 20

# #convert to equal area to set polygon size in metres
# gdf_poly = gdf.to_crs('EPSG:6933')

# #create circle buffer around points, then find envelope
# gdf_poly['geometry'] = gdf_poly['geometry'].buffer(radius).envelope

# #Convert back to lat/lon
# gdf_poly = gdf_poly.to_crs('EPSG:4326')

# #shuffle so labels aren't sequential
# gdf_poly = gdf_poly.reset_index(drop=True).drop(columns=['lon', 'lat'])

# #add plotid for CEO
# gdf_poly['PLOTID'] = range(1,len(gdf_poly)+1)
# gdf_poly['SAMPLEID'] = range(1,len(gdf_poly)+1)

# gdf_poly.to_file(folder + 'gfsad_divergence_samples_poly.shp')